In [1]:
from SnakeBoard import SnakeBoard
from SnakeGame import SnakeGame
from NeuralNetwork import NeuralNetwork
import numpy as np
import time
import matplotlib.pyplot as plt

# ---------- User defined parameters ----------

# Miscellaneous parameters
manual_play = 0 # Get user input (keyboard) instead of neural network auto-play
t_between_gen = 0 # Time (secs) between generations
n_gens_2_save_weights = 25 # Num of generations elapsed to save weights in a file

# Training parameters
n_of_gens = 5000 # Number of training generations
n_games_per_gen = 625 # Number of parallel games per generation
selected_games_per_gen = 15 # Selected baselines per generation to be used as references for mutations

# Mutation parameters
mrate_bias = 0.2
mrate_weights = 0.2
msize_bias = 0.2
msize_weights = 0.2

# ---------- Machine Learning main logic ---------- 

# Create game + ANN instances
[record_w_score, record_score] = [0, 0]
s_board = SnakeBoard(n_games_per_gen)
s_games, s_ann = [] , []
for i in range(n_games_per_gen):
    s_games.append(SnakeGame(s_board))
    s_ann.append(NeuralNetwork())

s_board.init_board()

# Run number of generations
best_score_history = list()
best_ann_weights_history = list()
for idx_gen in range(n_of_gens):

    # Step all games in current generation (until all games are over)
    while True:     
        game_status = list() 
        for idx_game, game in enumerate(s_games):
            
            # Get current game state and decide the next move
            state = game.get_game_state()
            if manual_play == 1:
                next_move = game.get_key()
            else:
                next_move = s_ann[idx_game].calculate(state) 
                if next_move == 0: next_move = "IDLE"
                elif next_move == 1: next_move = "T_LEFT"
                elif next_move == 2: next_move = "T_RIGHT"

            # Step game instance based on ANN calc. next move
            [game_over, w_score, score] = game.step_game(next_move)

            # Save game data in game status array
            game_status.append([game_over, w_score, idx_game, score])       

        # Update graphics of all games (visual feedback)
        #s_board.clear_board()
        #s_board.update_board_elements(s_games)

        # If all game instances are done, finish current generation
        game_over_list = [g[0] for g in game_status]
        if np.min(game_over_list)==True:
            break
    
    # Get best score + ANN in prev. generation
    game_status.sort(key=lambda x:x[1],reverse =True)
    game_status_best = game_status[0]
    print("GEN ", idx_gen, " ----- BEST SCORE: ", game_status_best[3] , " / ", game_status_best[1], " ----- RECORD: ", record_w_score, " / ", record_score)
    best_score_history.append(game_status_best[1])
    best_ann_weights_history.append(s_ann[game_status_best[2]])

    # Save weights in an external file
    if np.mod(idx_gen,n_gens_2_save_weights)==0:
        np.save("./training_history.npy",[best_ann_weights_history, best_score_history])
    
    # If best score in curr. generation is a record, show it
    if game_status_best[1] > record_w_score:
        record_w_score = game_status_best[1]
    if game_status_best[3] > record_score:
        record_score = game_status_best[3]
    
    # Get the best "selected_games_per_gen" games in the current generation
    # and place them in the first positions
    for i in range(selected_games_per_gen):
        s_ann[i] = s_ann[game_status[i][2]].copy()
    
    # Mutate the best ones in the subsequent positions
    for i in range(selected_games_per_gen, n_games_per_gen):
        s_ann[i] = s_ann[np.mod(i,selected_games_per_gen)].copy()
        s_ann[i].mutate(mrate_weights,msize_weights, mrate_bias, msize_bias) # random mutations

    #Reset all games once they're finished
    for idx_game, game in enumerate(s_games):
        game.reset_game()
    
    time.sleep(t_between_gen)

s_board.quit_board()

pygame 2.5.2 (SDL 2.28.3, Python 3.9.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
SnakeBoard instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance created.
SnakeGame instance 

In [ ]:
#plt.plot(best_score_history)